# Data Science HW2

## 【檔案說明】

### data.csv
id 為每項 data 的編號，每項 data 總共有 19 個 attributes，全部有 2100 筆資料
### test.csv
在此文件內，顯示的是我們需要比較是否為相同 cluster 的
data 編號，例如 index 0 要比較的資料為 id 1303 以及 id
1234。
### submit.csv
需要將預測結果 ( 0 : 不為同一群 / 1 : 同一群 ) 寫進
submit.csv 內，也就是在 index 0 的 ans 需要把 id 1303 & id
1234 是否為同一群的結果寫入，同理 index 1 的 ans 需要寫入
id 1710 & id 878 是否為同一群的結果，以此類推，總共要預測
400 筆資料。

## 【資料集說明】

**目標：透過 19 個 attributes 去分析各個 data 是否為同一群。**


在此 dataset 中，每筆 id(rows)代表一張圖片，而一張圖片通常可以用多個屬性(columns)集合來表示，以下為各屬性所代表的意思，請利用這些屬性將資料做分群(也就是同一群內的圖片具有較相似的屬性)。

[註: line extraction algorithm 是在做圖像分析時會用到的特徵萃取演算法]

| Column 1 | Column 2 |
| -------- | -------- |
| Feature1-2: | 該圖片中物件的中心點座標值 |
| Feature3: | 將圖片預先分成九個區塊，故皆為 9 |
| Feature4: | line extraction 的結果中低對比且長度小於等於 5 的區域則為 1，並且正規化 |
| Feature5: | line extraction 的結果中低對比且長度大於 5 的區域則為 1，並且正規化 |
| Feature6: | 該區域在水平鄰近像素對比度的平均值 |
| Feature7: | 該區域在水平鄰近像素對比度的標準差 |
| Feature8: | 該區域在垂直鄰近像素對比度的平均值 |
| Feature9: | 該區域在垂直鄰近像素對比度的標準差 |
| Feature10: | 每個區域的密度平均值或所謂的灰階值，公式為(R+B+G)/3 |
| Feature11-13: | RGB 在該區域各自的平均值 |
| Feature14-16: | RGB 在該區域各自的 excess value，公式分別為(2R-G-B)、(2B-G-R)、(2G-R-B) |
| Feature17-19: | 分別為明度、飽和度、色相的平均值 |

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering 
from sklearn.decomposition import PCA
from sklearn import metrics

In [2]:
Data = pd.read_csv('data.csv')
Test = pd.read_csv('test.csv')

In [3]:
Data

,id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19
0,0,97,123,9,0.000000,0.000000,1.555555,1.096294,13.222221,108.740740,52.814816,48.000000,64.111115,46.333332,-14.444445,33.888890,-19.444445,64.111115,0.280593,-1.994105
1,1,37,189,9,0.000000,0.000000,1.388889,1.485184,5.722223,23.885176,30.000000,27.111110,36.333336,26.555555,-8.666667,19.000000,-10.333333,36.333336,0.265248,-2.050282
2,2,148,169,9,0.111111,0.000000,1.055555,0.774073,1.833333,2.611110,19.592592,15.111111,18.666668,25.000000,-13.444445,-2.777778,16.222221,25.000000,0.396121,2.467752
3,3,95,57,9,0.000000,0.000000,1.833333,3.411111,2.111111,1.718517,26.296297,24.666666,34.444447,19.777779,-4.888889,24.444445,-19.555555,34.444447,0.425691,-1.739017
4,4,200,189,9,0.000000,0.000000,1.111111,1.088662,3.166667,1.005541,49.666668,45.111110,60.888890,43.000000,-13.666667,33.666668,-20.000000,60.888890,0.293810,-1.973221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,2095,25,91,9,0.000000,0.000000,1.611111,4.374074,1.388889,0.996296,3.888889,0.888889,8.111112,2.666667,-9.000000,12.666667,-3.666667,8.111112,0.911448,-2.354575
2096,2096,178,185,9,0.000000,0.000000,2.611112,1.806368,3.111111,2.277101,49.037037,44.000000,60.666668,42.444443,-15.111111,34.888890,-19.777779,60.666668,0.300036,-2.006186
2097,2097,237,135,9,0.000000,0.000000,0.555556,0.207407,0.666667,0.355555,4.148148,5.666666,4.777778,2.000000,4.555555,1.888889,-6.444445,5.666666,0.647222,-0.807848
2098,2098,103,39,9,0.000000,0.000000,0.499997,0.299997,1.111111,0.429627,127.518520,116.555560,142.888890,123.111115,-32.888890,46.111110,-13.222222,142.888890,0.184263,-2.352961


In [4]:
# Drop the RPG stuff, used grey scale
Test = Test.drop(['index'], axis=1)
ID = Data['id']

# feature45 = pd.DataFrame(Data, columns = ['feature4','feature5'])

Data = Data.drop(['id',
#                  'feature1','feature2',
                 'feature3',
                  'feature4','feature5',
                  'feature11','feature12','feature13',
#                  'feature14','feature15','feature16',
                  'feature17', 'feature18','feature19'
                 ],axis = 1)

Data

,feature1,feature2,feature6,feature7,feature8,feature9,feature10,feature14,feature15,feature16
0,97,123,1.555555,1.096294,13.222221,108.740740,52.814816,-14.444445,33.888890,-19.444445
1,37,189,1.388889,1.485184,5.722223,23.885176,30.000000,-8.666667,19.000000,-10.333333
2,148,169,1.055555,0.774073,1.833333,2.611110,19.592592,-13.444445,-2.777778,16.222221
3,95,57,1.833333,3.411111,2.111111,1.718517,26.296297,-4.888889,24.444445,-19.555555
4,200,189,1.111111,1.088662,3.166667,1.005541,49.666668,-13.666667,33.666668,-20.000000
...,...,...,...,...,...,...,...,...,...,...
2095,25,91,1.611111,4.374074,1.388889,0.996296,3.888889,-9.000000,12.666667,-3.666667
2096,178,185,2.611112,1.806368,3.111111,2.277101,49.037037,-15.111111,34.888890,-19.777779
2097,237,135,0.555556,0.207407,0.666667,0.355555,4.148148,4.555555,1.888889,-6.444445
2098,103,39,0.499997,0.299997,1.111111,0.429627,127.518520,-32.888890,46.111110,-13.222222


In [5]:
Data.isnull().sum()

feature1     0
feature2     0
feature6     0
feature7     0
feature8     0
feature9     0
feature10    0
feature14    0
feature15    0
feature16    0
dtype: int64

## Normalize

In [6]:
X= np.asarray(Data)
from sklearn.preprocessing import StandardScaler, Normalizer, scale

scale = StandardScaler()
X = scale.fit_transform(X)
# normalizer = Normalizer().fit(X)
# X = normalizer.transform(X)

In [7]:
X.shape

(2100, 10)

## Reducing the dimensions of the data 

In [8]:
# pca = PCA(n_components = 9)
# X = pca.fit_transform(X)
# X = pd.DataFrame(X)
# # X.columns = ['P1', 'P2']

# X.head(2) 

In [9]:
kmean = KMeans(n_clusters=5, random_state = 0).fit(X)
pred = kmean.labels_
print(pred)

[2 2 1 ... 0 3 1]


## Match if they were same cluster

In [10]:
result = np.array([],dtype=int)

for index,value in enumerate(Test.values):
#     print(index,value)
    if pred[value[0]] == pred[value[1]]:
        result = np.append(result,1)
#         print('yes')
    else:
        result = np.append(result,0)
#         print('no')


## 輸出 submit.csv

In [11]:
def output_csv(result, filename):
    df = pd.DataFrame(result)
    df.to_csv(filename, header = ['ans'], index_label = 'index')

output_csv(result, "submit.csv")